# Datenvorbereitung

In [ ]:
import pandas as pd
import networkx as nx
import random
import numpy as np
import os

out_dir = "data"
in_dir = "{}/followers".format(out_dir)
dfs = []

for dirent in os.listdir(in_dir):
    path = os.path.join(in_dir, dirent)

    if not os.path.isfile(path) or not dirent.endswith(".csv") or "_lvl2" in dirent or "_friends" in dirent:
        continue

    print(dirent)
    dfs.append(pd.read_csv(path, lineterminator='\n'))

df_hochschul_friends = pd.read_csv(in_dir + "/hochschulen_friends.csv")
df_hochschul_friends_reverse = df_hochschul_friends.rename(columns={"id": "follower_of", "friend_of": "id"})
df = pd.concat(dfs)
df = pd.concat([df, df_hochschul_friends_reverse])

df[["id", "follower_of"]].sort_values("id", ascending=True).to_csv(
    "{}/direct_followings.csv".format(out_dir),
    index=False,
)

static_df = pd.read_csv("{}/static_users.csv".format(out_dir))

static_df['follower_of'] = static_df['id']

df_hochschul_friends = df_hochschul_friends.rename(columns={"friend_of": "follower_of"})

pd.concat([
    static_df,
    df,
    df_hochschul_friends,
]).drop_duplicates(["id"]).to_csv(
    "{}/direct_users.csv".format(out_dir),
    index=False,
)

hsduesseldorf_lvl2 = pd.read_csv('data/followers/hsduesseldorf_lvl2.csv')

hsduesseldorf_lvl2[["id", "follower_of"]].sort_values("id", ascending=True).to_csv(
    "{}/followings_lvl2.csv".format(out_dir),
    index=False,
)

hsduesseldorf_lvl2.drop_duplicates(["id"]).to_csv(
    "{}/users_lvl2.csv".format(out_dir),
    index=False,
)

In [27]:
import pandas as pd
import networkx as nx
import random
import numpy as np

lvl2_users_hsd = pd.read_csv('data/users_lvl2.csv')
lvl2_edges_hsd = pd.read_csv('data/followings_lvl2.csv')
nodes = pd.read_csv('data/direct_users.csv')
edges = pd.read_csv('data/direct_followings.csv')
nodes = pd.concat([nodes, lvl2_users_hsd]).drop_duplicates(["id"])
edges = pd.concat([edges, lvl2_edges_hsd])

nodes['viz'] = [{'color':{'r':255, 'g':255, 'b':255, 'a':1}}] * len(nodes)
nodes['gender'] = ["unknown"] * len(nodes)
hochschul_ids = [103823788, 84606793, 124155166, 265859722, 2776187059, 928008620, 11053712, 3063800235]

nodes = nodes[nodes['id'].notna()]

C:\Users\Lemjoe\AppData\Local\Temp\ipykernel_12388\1585940841.py:6: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  lvl2_users_hsd = pd.read_csv('data/users_lvl2.csv')
C:\Users\Lemjoe\AppData\Local\Temp\ipykernel_12388\1585940841.py:8: DtypeWarning: Columns (7,13,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  nodes = pd.read_csv('data/direct_users.csv')


In [28]:
nodes = nodes.drop(columns=['withheld_in_countries','url','follower_of', 'created_at', 'has_extended_profile','default_profile','default_profile_image'])
nodes = nodes.astype({'description': 'string', 'screen_name': 'string'})

In [29]:
import re

females = []
male = []
for index, row in nodes.iterrows():
    
    if pd.notnull(row['description']):
        isFemale = re.search(r'\bshe/her\b|\bSie/Ihr\b', row['description'])
        if isFemale != None:
            females.append(row['id'])
        isMale = re.search(r'\bhe/him\b|\bEr/Ihn\b', row['description'])
        if isMale != None:
            male.append(row['id'])

print(len(females))
print(len(male))


6470
3966


In [30]:
nodes.loc[nodes['id'].isin(females),('gender')] = 'female'
nodes.loc[nodes['id'].isin(male), ('gender') ]= 'male'

In [31]:
nodes = nodes.drop(columns=['description'])
nodes['name'] =nodes['name'].str.replace(r'\x08|\x1f|\x1e', '')
nodes['location'] =nodes['location'].str.replace(r'\x08|\x1f|\x1e', '')

C:\Users\Lemjoe\AppData\Local\Temp\ipykernel_12388\1549860975.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  nodes['name'] =nodes['name'].str.replace(r'\x08|\x1f|\x1e', '')
C:\Users\Lemjoe\AppData\Local\Temp\ipykernel_12388\1549860975.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  nodes['location'] =nodes['location'].str.replace(r'\x08|\x1f|\x1e', '')


In [32]:
random.seed(10)

def get_random_Color_hex():
    return {'r' : random.randint(0,255),'g' : random.randint(0,255), 'b' : random.randint(0,255), 'a':1}

def lerp(a, b, t):
    return a*(1 - t) + b*t

# Louvain

In [ ]:
graph = nx.from_pandas_edgelist(edges, source='id', target='follower_of', create_using=nx.Graph())

node_attr = nodes.set_index('id').to_dict('index')
nx.set_node_attributes(graph, node_attr)

In [8]:
import networkx.algorithms.community as nx_comm

communities = nx_comm.louvain_communities(graph, seed=123)
len(communities)

69

In [9]:
for community in communities:
    color = get_random_Color_hex()
    for id in list(community):
        update = {id: {"viz": {'color':color}}}
        nx.set_node_attributes(graph, update)

In [10]:
for edge in graph.edges():
    color1 = graph.nodes[edge[0]]["viz"]['color']
    color2 = graph.nodes[edge[1]]["viz"]['color']
    color1 = np.array([color1['r'], color1['g'], color1['b']])
    color2 = np.array([color2['r'], color2['g'], color2['b']])
    edge_color = lerp(np.array(color1), np.array(color2), 0.5)
    edge_color = edge_color.astype(int)
    update = {edge: {"viz":{'color': {'r':edge_color[0], 'g':edge_color[1], 'b':edge_color[2],'a':1}}}}
    nx.set_edge_attributes(graph, update)

In [11]:
nx.write_gexf(graph, path="./komplette_twitter_daten_louvain_zuteilung.gexf")

# Gerichteten Graph mit Knoten Färbung

In [11]:
graph = nx.from_pandas_edgelist(edges, source='id', target='follower_of', create_using=nx.DiGraph())

node_attr = nodes.set_index('id').to_dict('index')
nx.set_node_attributes(graph, node_attr)

In [12]:
amount_of_degrees = np.array(sorted(list(set([degree for _, degree in graph.degree()]))))
amount_of_degrees_small_nodes = amount_of_degrees[(amount_of_degrees < 100) & (amount_of_degrees > 1)]
amount_of_degrees_large_nodes =  amount_of_degrees[amount_of_degrees >= 100]

In [13]:
large_community_ids = []

In [14]:
for node, degree in graph.degree():
    if degree >= 100:
        update = {node: {"viz": {'color':get_random_Color_hex()}}}
        nx.set_node_attributes(graph, update)
        large_community_ids.append(node)

In [15]:
out_nodes_all = [node for (node, val) in graph.out_degree() if val == 1]
in_nodes_all = [node for (node, val) in graph.in_degree() if val == 1]
out_edges_all = np.array([[src, dest] for node in out_nodes_all for src, dest in list(graph.out_edges(node))])
in_edges_all = np.array([[src, dest] for node in in_nodes_all for src, dest in list(graph.in_edges(node))])


In [16]:
for ids in large_community_ids:
    out_nodes = out_edges_all[out_edges_all[:, 1]== ids]
    in_nodes = in_edges_all[in_edges_all[:, 0] == ids]

    one_degree_nodes = np.concatenate([out_nodes[:, 0], in_nodes[:, 1]])
    updated_nodes = [ {node: {"viz": {'color':graph.nodes[ids]["viz"]['color']}}} for node in one_degree_nodes]
    for update in updated_nodes:
        nx.set_node_attributes(graph, update)


In [17]:
for degree in amount_of_degrees_small_nodes:
    for out_degree, in_degree in zip(graph.out_degree(), graph.in_degree()):

        if in_degree[1] == degree:
            src = [src for src, _ in list(graph.in_edges(in_degree[0]))]
            colors = [graph.nodes[id]["viz"]['color'] for id in src]
            mixed = graph.nodes[in_degree[0]]["viz"]['color']
            mixed = np.array([mixed['r'], mixed['g'], mixed['b']])
            for color in colors:
                target = np.array([color['r'], color['g'], color['b']])
                mixed = lerp(mixed, target, 0.25)
            mixed = mixed.astype(int)
            update = {in_degree[0]: {"viz": {'color': {'r':mixed[0], 'g':mixed[1], 'b':mixed[2], 'a':1}}}}
            nx.set_node_attributes(graph, update)

        if out_degree[1] == degree:
            dest = [dest for _, dest in list(graph.out_edges(out_degree[0]))]
            colors = [graph.nodes[id]["viz"]['color'] for id in dest]
            mixed = graph.nodes[out_degree[0]]["viz"]['color']
            mixed = np.array([mixed['r'], mixed['g'], mixed['b']])
            for color in colors:
                target = np.array([color['r'], color['g'], color['b']])
                mixed = lerp(mixed, target, 0.5)
            mixed = mixed.astype(int)
            update = {out_degree[0]: {"viz": {'color': {'r':mixed[0], 'g':mixed[1], 'b':mixed[2], 'a':1}}}}
            nx.set_node_attributes(graph, update)
                

In [ ]:
for edge in graph.edges():
    color1 = graph.nodes[edge[0]]["viz"]['color']
    color2 = graph.nodes[edge[1]]["viz"]['color']
    color1 = np.array([color1['r'], color1['g'], color1['b']])
    color2 = np.array([color2['r'], color2['g'], color2['b']])
    edge_color = lerp(np.array(color1), np.array(color2), 0.5)
    edge_color = edge_color.astype(int)
    update = {edge: {"viz":{'color': {'r':edge_color[0], 'g':edge_color[1], 'b':edge_color[2],'a':1}}}}
    nx.set_edge_attributes(graph, update)

In [ ]:
nx.write_gexf(graph, path="./komplette_twitter_daten.gexf")

# Nodes entfernen

In [33]:
graph = nx.from_pandas_edgelist(edges, source='id', target='follower_of', create_using=nx.DiGraph())

node_attr = nodes.set_index('id').to_dict('index')
nx.set_node_attributes(graph, node_attr)

In [34]:
out_degree_nodes = [node for (node, val) in graph.out_degree() if val == 1 and graph.in_degree(node) == 0]
edges_one_degree = edges[edges['id'].isin(out_degree_nodes)]
edges_one_degree = edges_one_degree.sort_values("follower_of", ascending=True)
print(len(out_degree_nodes))
master_nodes = edges_one_degree.follower_of.unique()

879762


In [35]:
nodes_id_to_unify = []
for node in master_nodes:
    nodes_id_to_unify.append(edges_one_degree[edges_one_degree['follower_of'] == node]['id'].values.tolist()) 
nodes['number_of_nodes'] = 1

In [36]:
nodes_copy = nodes.copy(deep=False)
edges_copy = edges.copy(deep=False)

In [37]:
unified_node_id = -1
for list_of_nodes,master_node in zip(nodes_id_to_unify, master_nodes):
    
    new_node = pd.DataFrame({
        'id':unified_node_id,
        'id_str':''            ,  
        'name':'follower_of_'+str(master_node), 
        'screen_name': 'follower_of_'+nodes[nodes['id']==master_node]['screen_name'].values[0], 
        'location':'', 
        'protected':'False', 
        'followers_count':0,
        'friends_count':0,
        'listed_count':0,
        'favourites_count':0,
        'verified':'False', 
        'statuses_count':0,
        'viz':{'color':nodes[nodes['id']==master_node]['viz'].values[0]}, 
        'gender':'unknown', 
        'number_of_nodes': len(list_of_nodes) })
    nodes_copy = pd.concat([nodes_copy,new_node], ignore_index=True)
    new_edge = pd.DataFrame({
        'id':[unified_node_id],
        'follower_of': [master_node] })
    edges_copy = pd.concat([edges_copy,new_edge], ignore_index=True)
    unified_node_id -= 1



In [38]:
flatten_node_to_remove = [node for list_of_nodes in nodes_id_to_unify for node in list_of_nodes]
nodes_copy = nodes_copy[~nodes_copy['id'].isin(flatten_node_to_remove)]
edges_copy = edges_copy[~edges_copy['id'].isin(flatten_node_to_remove)]

nodes_copy = nodes_copy.astype({'screen_name': 'string', 'id_str': 'string'})

In [39]:
print(len(nodes))
print(len(edges))
print(len(nodes_copy))
print(len(edges_copy))

1097880
1752555
219608
867543


In [40]:
graph = nx.from_pandas_edgelist(edges_copy, source='id', target='follower_of', create_using=nx.DiGraph())

node_attr = nodes_copy.set_index('id').to_dict('index')
nx.set_node_attributes(graph, node_attr)

In [41]:
for node, degree in graph.degree():
    if degree >= 100:
        update = {node: {"viz": {'color':get_random_Color_hex()}}}
        nx.set_node_attributes(graph, update)
        large_community_ids.append(node)

In [42]:
amount_of_degrees = np.array(sorted(list(set([degree for _, degree in graph.degree()]))))
amount_of_degrees_small_nodes = amount_of_degrees[(amount_of_degrees < 100) & (amount_of_degrees > 1)]
amount_of_degrees_large_nodes =  amount_of_degrees[amount_of_degrees >= 100]

In [43]:
out_nodes_all = [node for (node, val) in graph.out_degree() if val == 1]
in_nodes_all = [node for (node, val) in graph.in_degree() if val == 1]
out_edges_all = np.array([[src, dest] for node in out_nodes_all for src, dest in list(graph.out_edges(node))])
in_edges_all = np.array([[src, dest] for node in in_nodes_all for src, dest in list(graph.in_edges(node))])

In [44]:
large_community_ids = []
for ids in large_community_ids:
    out_nodes = out_edges_all[out_edges_all[:, 1]== ids]
    in_nodes = in_edges_all[in_edges_all[:, 0] == ids]

    one_degree_nodes = np.concatenate([out_nodes[:, 0], in_nodes[:, 1]])
    updated_nodes = [ {node: {"viz": {'color':graph.nodes[ids]["viz"]['color']}}} for node in one_degree_nodes]
    for update in updated_nodes:
        nx.set_node_attributes(graph, update)

In [45]:
for degree in amount_of_degrees_small_nodes:
    for out_degree, in_degree in zip(graph.out_degree(), graph.in_degree()):

        if in_degree[1] == degree:
            src = [src for src, _ in list(graph.in_edges(in_degree[0]))]
            colors = [graph.nodes[id]["viz"]['color'] for id in src]
            mixed = graph.nodes[in_degree[0]]["viz"]['color']
            mixed = np.array([mixed['r'], mixed['g'], mixed['b']])
            for color in colors:
                target = np.array([color['r'], color['g'], color['b']])
                mixed = lerp(mixed, target, 0.25)
            mixed = mixed.astype(int)
            update = {in_degree[0]: {"viz": {'color': {'r':mixed[0], 'g':mixed[1], 'b':mixed[2], 'a':1}}}}
            nx.set_node_attributes(graph, update)

        if out_degree[1] == degree:
            dest = [dest for _, dest in list(graph.out_edges(out_degree[0]))]
            colors = [graph.nodes[id]["viz"]['color'] for id in dest]
            mixed = graph.nodes[out_degree[0]]["viz"]['color']
            mixed = np.array([mixed['r'], mixed['g'], mixed['b']])
            for color in colors:
                target = np.array([color['r'], color['g'], color['b']])
                mixed = lerp(mixed, target, 0.5)
            mixed = mixed.astype(int)
            update = {out_degree[0]: {"viz": {'color': {'r':mixed[0], 'g':mixed[1], 'b':mixed[2], 'a':1}}}}
            nx.set_node_attributes(graph, update)
                

In [46]:
for edge in graph.edges():
    color1 = graph.nodes[edge[0]]["viz"]['color']
    color2 = graph.nodes[edge[1]]["viz"]['color']
    color1 = np.array([color1['r'], color1['g'], color1['b']])
    color2 = np.array([color2['r'], color2['g'], color2['b']])
    edge_color = lerp(np.array(color1), np.array(color2), 0.5)
    edge_color = edge_color.astype(int)
    update = {edge: {"viz":{'color': {'r':edge_color[0], 'g':edge_color[1], 'b':edge_color[2],'a':1}}}}
    nx.set_edge_attributes(graph, update)

In [47]:
nx.write_gexf(graph, path="./komplette_twitter_daten_vereinfacht2.gexf")